## Crack detection model using keras resnet50


In [1]:
# !wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

# 1. Libraries Setup

In [3]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16_input


## 2.1 Defining Parameters

In [4]:
params = {
    'image_size': 224,
    'batch_size': 100,
    'num_epochs': 2,
    'class_mode': 'categorical',
    'num_classes': 2,
    'shuffle': True}

## 2.2. Defining Useful Functions

In [5]:
def batch_generator(path, preprocess_input, params):
    data_generator = ImageDataGenerator(
        preprocessing_function=preprocess_input)
    
    generator = data_generator.flow_from_directory(
        path,
        target_size=(params['image_size'], 
                     params['image_size']),
        batch_size=params['batch_size'],
        class_mode=params['class_mode'],
        shuffle=params['shuffle'])
    
    return generator


# 3. Create Generators 

In [6]:
train_generator = batch_generator(
    '../input/concrete-data-wk4/concrete_data_week4/train',
    preprocess_vgg16_input,
    params)
validation_generator = batch_generator(
    '../input/concrete-data-wk4/concrete_data_week4/valid',
    preprocess_vgg16_input,
    params)
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


# 4. Create DeepLearning Model

In [7]:
model = keras.Sequential()

model.add(VGG16(weights = 'imagenet', include_top = False, pooling = 'avg'))
model.layers[0].trainable = False

model.add(Dense(params['num_classes'], activation = 'softmax'))

model.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


# 5. Compile the Model

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### **NOTE**: 
The model is not run here, it was trained in a previous session. The model is reloaded in Section 2.

In [9]:
%%script false --no-raise-error
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=params['num_epochs'],
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

In [10]:
%%script false --no-raise-error
model.save('classifier_vgg16_model.h5')

# 1. Load models back

In [11]:
resnet50 = keras.models.load_model('../input/classifier-resnet50/classifier_resnet_model.h5')
vgg16 = keras.models.load_model('../input/classifier-vgg16/classifier_vgg16_model.h5')
print(resnet50.summary())
print(vgg16.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________
None
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 512)               14714688  
_________________________

# 2. Data Generator

See the function used ```batch_generator``` in the first section of the notebook.

In [12]:
params['shuffle'] = False
test_resnet_generator = batch_generator(
    '../input/concrete-data-wk4/concrete_data_week4/test',
    preprocess_resnet50_input, params)
test_vgg16_generator = batch_generator(
    '../input/concrete-data-wk4/concrete_data_week4/test',
    preprocess_vgg16_input, params)

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


# 3. Evaluate the models
The method ```evaluate_generator``` is actually deprecated. Currently, ```evaluate``` handles generators as argument.

In [13]:
result_resnet = resnet50.evaluate_generator(
    test_resnet_generator,
    verbose=1)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


5/5 [==============================] - 8s 466ms/step - loss: 0.0046 - accuracy: 1.0000


In [14]:
result_vgg16 = vgg16.evaluate_generator(
    test_vgg16_generator,
    verbose=1)

5/5 [==============================] - 101s 11s/step - loss: 0.0696 - accuracy: 0.9840


# 4. VGG16 Performance

In [15]:
print(f"Accuracy: {result_vgg16[1]}")
print(f"Loss: {result_vgg16[0]}")

Accuracy: 0.984000027179718
Loss: 0.06960207968950272


<a id="item44"></a>

# 5. ResNet50 Performance

In [16]:
print(f"Accuracy: {result_resnet[1]}")
print(f"Loss: {result_resnet[0]}")

Accuracy: 1.0
Loss: 0.004571572877466679


# 1. Make predictions

The method ```predict_generator``` is actually deprecated. Currently, ```predict``` handles generators as argument.

In [17]:
import numpy as np

def get_predictions(predictions):
    for pred in predictions:
        position = np.argmax(pred)
        if position == 1:
            print("Positive")
        if position == 0:
            print("Negative")

In [18]:
resnet_prediction = resnet50.predict_generator(
    test_resnet_generator,
    verbose=1)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


5/5 [==============================] - 3s 411ms/step


In [19]:
vgg16_prediction = vgg16.predict_generator(
    test_vgg16_generator,
    verbose=1)

5/5 [==============================] - 54s 13s/step


# 2. Class predictions report

In [20]:
get_predictions(resnet_prediction[:5])

Negative
Negative
Negative
Negative
Negative


In [21]:
get_predictions(vgg16_prediction[:5])

Negative
Negative
Negative
Negative
Negative


<h2>About the Authors:</h2> 

<a href="https://www.linkedin.com/in/abisek-kamthan-846b10243/">ABISEK KAMTHAN R S</a>
